In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col,sum,desc,count, row_number,lit,mean, format_number,rank,asc
from pyspark.sql.window import Window

In [4]:
spark = SparkSession.builder.appName("BD_Processing").getOrCreate()

In [5]:
#Se define el schema del dataset world-happiness-report
structSchema_df = StructType([ \
    StructField('Country name', StringType(), True), \
    StructField('year', StringType(), True), \
    StructField('Life Ladder', StringType(), True), \
    StructField('Log GDP per capita', StringType(), True), \
    StructField('Social support', StringType(), True), \
    StructField('Healthy life expectancy at birth', StringType(), True), \
    StructField('Freedom to make life choices', StringType(), True), \
    StructField('Generosity', StringType(), True), \
    StructField('Perceptions of corruption', StringType(), True), \
    StructField('Positive affect', StringType(), True), \
    StructField('Negative affect', StringType(), True) \
      ])

In [7]:
df = spark.read.schema(structSchema_df).load(r"C:\Users\jeffe\OneDrive\KeepCoding Bootcamp\BD_Processing_Homework\world-happiness-report.txt",format="csv", sep=",", header = "true")
df.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- Life Ladder: string (nullable = true)
 |-- Log GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy at birth: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Positive affect: string (nullable = true)
 |-- Negative affect: string (nullable = true)



In [8]:
df.show(10,False)

+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|Afghanistan |2008|3.724      |7.370             |0.451         |50.800                          |0.718                       |0.168     |0.882                    |0.518          |0.258          |
|Afghanistan |2009|4.402      |7.540             |0.552         |51.200                          |0.679                       |0.190     |0.850                    |0.584          |0.237          |
|Afghanistan |2

In [9]:
#Se define el schema del dataset world-happiness-report-2021
structSchema_df_21 = StructType([ \
    StructField('Country name', StringType(), True), \
    StructField('Regional indicator', StringType(), True), \
    StructField('Ladder score', StringType(), True), \
    StructField('Standard error of ladder score', StringType(), True), \
    StructField('upperwhisker', StringType(), True), \
    StructField('lowerwhisker', StringType(), True), \
    StructField('Logged GDP per capita', StringType(), True), \
    StructField('Social support', StringType(), True), \
    StructField('Healthy life expectancy', StringType(), True), \
    StructField('Freedom to make life choices', StringType(), True), \
    StructField('Generosity', StringType(), True), \
    StructField('Perceptions of corruption', StringType(), True), \
    StructField('Ladder score in Dystopia', StringType(), True), \
    StructField('Explained by: Log GDP per capita', StringType(), True), \
    StructField('Explained by: Social support', StringType(), True), \
    StructField('Explained by: Healthy life expectancy', StringType(), True), \
    StructField('Explained by: Freedom to make life choices', StringType(), True), \
    StructField('Explained by: Generosity', StringType(), True), \
    StructField('Explained by: Perceptions of corruption', StringType(), True), \
    StructField('Dystopia + residual', StringType(), True) \
      ])

In [10]:
df_21 = spark.read.schema(structSchema_df_21).load(r"C:\Users\jeffe\OneDrive\KeepCoding Bootcamp\BD_Processing_Homework\world-happiness-report-2021.txt",format="csv", sep=",", header = "true")
df_21.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: string (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Ladder score in Dystopia: string (nullable = true)
 |-- Explained by: Log GDP per capita: string (nullable = true)
 |-- Explained by: Social support: string (nullable = true)
 |-- Explained by: Healthy life expectancy: string (nullable = true)
 |-- Explained by: Freedom to make life choices: string (nullable = true)
 |-- Explained by: Generosity: string (nullable = true)
 |-- Explained 

In [11]:
df_21.show(10,False)

+------------+---------------------+------------+------------------------------+------------+------------+---------------------+--------------+-----------------------+----------------------------+----------+-------------------------+------------------------+--------------------------------+----------------------------+-------------------------------------+------------------------------------------+------------------------+---------------------------------------+-------------------+
|Country name|Regional indicator   |Ladder score|Standard error of ladder score|upperwhisker|lowerwhisker|Logged GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|Ladder score in Dystopia|Explained by: Log GDP per capita|Explained by: Social support|Explained by: Healthy life expectancy|Explained by: Freedom to make life choices|Explained by: Generosity|Explained by: Perceptions of corruption|Dystopia + residual|
+------------+------------

In [12]:
#Se castea la columna Ladder score como "double" debido a que en la estructura se definio como string
df_21 = df_21.withColumn("Ladder score dob", col("Ladder score").cast("double"))

In [13]:
#Como el data set del 2021 no tiene un campo del 2021 es necesario agregar una columna year con este valor
df_21 = df_21.withColumn("year", lit(2021))

In [14]:
df_21.show(10,False)

+------------+---------------------+------------+------------------------------+------------+------------+---------------------+--------------+-----------------------+----------------------------+----------+-------------------------+------------------------+--------------------------------+----------------------------+-------------------------------------+------------------------------------------+------------------------+---------------------------------------+-------------------+----------------+----+
|Country name|Regional indicator   |Ladder score|Standard error of ladder score|upperwhisker|lowerwhisker|Logged GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|Ladder score in Dystopia|Explained by: Log GDP per capita|Explained by: Social support|Explained by: Healthy life expectancy|Explained by: Freedom to make life choices|Explained by: Generosity|Explained by: Perceptions of corruption|Dystopia + residual|Ladde

<h1>PREGUNTA 1</h1>
<h4> ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score”
mayor número más feliz es el país)
</h4>


In [16]:
#La pregunta hace referencia al dataset del 2021 por lo que se usara este dataset
df_preg1 = df_21.orderBy(desc("Ladder score dob"))
df_preg1 = df_preg1.select( df_preg1['Country name'],df_preg1['Ladder score dob'])
df_preg1.show(1,False)

+------------+----------------+
|Country name|Ladder score dob|
+------------+----------------+
|Finland     |7.842           |
+------------+----------------+
only showing top 1 row



<h1>PREGUNTA 2</h1>
<h4>  ¿Cuál es el país más “feliz” del 2021 por continente según la data?
</h4>

In [18]:
#La pregunta hace referencia al dataset del 2021 por lo que se usara este dataset
df_preg2 = df_21.select( df_21['Country name'],df_21['Regional indicator'], df_21['Ladder score dob'])

window = Window.partitionBy('Regional indicator').orderBy(desc('Ladder score dob'))
df_preg2_highest = df_preg2.withColumn('row_number', row_number().over(window)).filter(col('row_number') == 1).drop('row_number')

df_preg2_highest = df_preg2_highest.select( df_21['Regional indicator'],df_21['Country name'], df_21['Ladder score dob'])

df_preg2_highest.show(15,False)

+----------------------------------+------------------------+----------------+
|Regional indicator                |Country name            |Ladder score dob|
+----------------------------------+------------------------+----------------+
|Central and Eastern Europe        |Czech Republic          |6.965           |
|Commonwealth of Independent States|Uzbekistan              |6.179           |
|East Asia                         |Taiwan Province of China|6.584           |
|Latin America and Caribbean       |Costa Rica              |7.069           |
|Middle East and North Africa      |Israel                  |7.157           |
|North America and ANZ             |New Zealand             |7.277           |
|South Asia                        |Nepal                   |5.269           |
|Southeast Asia                    |Singapore               |6.377           |
|Sub-Saharan Africa                |Mauritius               |6.049           |
|Western Europe                    |Finland         

<h1>PREGUNTA 3</h1>
<h4>  ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
</h4>

In [19]:
#Para resolver esta pregunta se debera unir ambos datasets para tener todo el historico de los datos
#Se castea el campo Life ladder a double
df = df.withColumn("Life Ladder dob", col("Life Ladder").cast("double"))

#Se renombra campos del dataset del 2021
df_21 = df_21.withColumnRenamed("Ladder score","Life Ladder").withColumnRenamed("Logged GDP per capita","Log GDP per capita").withColumnRenamed("Healthy life expectancy","Healthy life expectancy at birth")

In [20]:
#Para realizar la union es indispensable que ambos datasets tenga la misma cantidad de columnas
for column in [column for column in df.columns if column not in df_21.columns]:
    df_21 = df_21.withColumn(column, lit(None))

for column in [column for column in df_21.columns if column not in df.columns]:
    df = df.withColumn(column, lit(None))

df_preg3 = df.union(df_21).distinct()

In [22]:
#Seleccionamos solo los campos que nos interesan para responder la pregunta:
df_preg3 = df_preg3.select("Country name","year","Life Ladder","Log GDP per capita","Healthy life expectancy at birth")

In [24]:
#para realizar las agrupaciones se emplea ventanas (windows) los cuales permiten agrupar por categorias
window_2 = Window.partitionBy('year').orderBy(desc('Life Ladder'))

df_preg3_highest = df_preg3.withColumn('row_number', row_number().over(window_2)).filter(col('row_number') == 1).drop('row_number')


In [25]:
#realizamos el conteo de todos los Numeros 1 y de manera adicional se calcula el promedio 
count_df = df_preg3_highest.groupBy("Country name").agg(count("*").alias("Total Nro 1"), format_number(mean("Life Ladder"), 2).alias("Avg Life Ladder dob"))
count_df = count_df.orderBy(desc("Total Nro 1"))
count_df.show()

+--------------------+-----------+-------------------+
|        Country name|Total Nro 1|Avg Life Ladder dob|
+--------------------+-----------+-------------------+
|             Denmark|          7|               7.80|
|             Finland|          7|               7.78|
|           Singapore|          1|               6.38|
|              Norway|          1|               7.60|
|Taiwan Province o...|          1|               6.58|
|              Canada|          1|               7.59|
|              Israel|          1|               7.16|
|               Nepal|          1|               5.27|
|         Switzerland|          1|               7.78|
|          Uzbekistan|          1|               6.18|
|      Czech Republic|          1|               6.96|
|         New Zealand|          1|               7.28|
|          Costa Rica|          1|               7.07|
|           Mauritius|          1|               6.05|
+--------------------+-----------+-------------------+



<h1>PREGUNTA 4</h1>
<h4>   ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?
</h4>

In [27]:
#Se menciona el year 2020 por lo que se puede utilizar el dataset correspondiente a este 
df_2020 = df.filter(col("year") == "2020")

#Se ordena el data set por Log GDP per capita
df_2020_ranked = df_2020.orderBy(desc("Log GDP per capita"))

#Como la columna es string es necesario castearlo a double
df_2020_ranked = df_2020_ranked.withColumn("Log GDP per capita dob", col("Log GDP per capita").cast("double"))

In [28]:
#Se agrupa por Life Ladder dob y se procede a realizar el ranking
window_3 = Window.orderBy(desc("Life Ladder dob"))
df_2020_ranked = df_2020_ranked.withColumn("Rank", rank().over(window_3))

In [29]:
#Se seleccionan los campos importantes para esta pregunta
df_preg4 = df_2020_ranked.select( df_2020_ranked['Country name'],df_2020_ranked['year'], df_2020_ranked['Log GDP per capita dob'],df_2020_ranked['Rank'])

df_preg4 = df_preg4.orderBy(desc("Log GDP per capita dob"))

df_preg4.show(1)

+------------+----+----------------------+----+
|Country name|year|Log GDP per capita dob|Rank|
+------------+----+----------------------+----+
|     Ireland|2020|                11.323|  13|
+------------+----+----------------------+----+
only showing top 1 row



<h1>PREGUNTA 5</h1>
<h4>    ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó 
o disminuyó?
</h4>

In [31]:
#Calculamos el promedio del GDP del 2020 para ello se filtra en la dataset con este year
df_2020 = df.filter(col("year") == "2020")

#Se calcula el promedio del campo "Log GDP per capita"
promedio_log_gdp_2020 = df_2020.select(mean("Log GDP per capita")).collect()[0][0]


In [34]:
#Calculamos el promedio dle GDP del 2021 para ello se emplea el dataset con este year
#Se castea el campo de "Logged GDP per capita" a doubletype debido a que en el schema se definio como string
df_21 = df_21.withColumn("Log GDP per capita", df_21["Log GDP per capita"].cast(DoubleType()))

#Se calcula el promedio del campo "Loggeg GDP per capita"
promedio_log_gdp_2021 = df_21.select(mean("Log GDP per capita")).collect()[0][0]

In [36]:
#Se calcula la variacion solicitada
variacion_porcentual = ((promedio_log_gdp_2021 - promedio_log_gdp_2020) / promedio_log_gdp_2020) * 100
print("La variación porcentual entre el promedio de 'Logged GDP per capita' en 2020 y 2021 es:", round(variacion_porcentual, 2), "%")

La variación porcentual entre el promedio de 'Logged GDP per capita' en 2020 y 2021 es: -3.27 %


<h1>PREGUNTA 6</h1>
<h4>    ¿Cuál es el país con mayor expectativa de vide (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia 
en ese indicador en el 2019?
</h4>

In [42]:
#Como se debe compara todos los years entonces se debe utilizar los dataset unidos. En la pregunta 3
#se define este dataset. Ademas se castea el campo "Healthy life expectancy at birth"
df_preg6 = df_preg3.withColumn("Healthy life expectancy at birth", df["Healthy life expectancy at birth"].cast(DoubleType()))

#Ordenamos el dataframe por el campo "Healthy life expectancy at birth"
df_preg6 = df_preg6.orderBy(desc("Healthy life expectancy at birth"))

#Se mues
df_preg6.show(5,False)

+------------+----+-----------+------------------+--------------------------------+
|Country name|year|Life Ladder|Log GDP per capita|Healthy life expectancy at birth|
+------------+----+-----------+------------------+--------------------------------+
|Singapore   |2019|6.378      |11.486            |77.1                            |
|Singapore   |2018|6.375      |11.490            |76.8                            |
|Singapore   |2017|6.378      |11.461            |76.5                            |
|Singapore   |2016|6.033      |11.419            |76.2                            |
|Singapore   |2015|6.620      |11.400            |75.9                            |
+------------+----+-----------+------------------+--------------------------------+
only showing top 5 rows



In [43]:
#Se sabe que el pais debido al resultado anterior es singapore por lo que ahora se debe realizar el filtrado
#de ese pais y tambien el year que se consulta
df_preg6 = df_preg6.filter((col("year") == "2019") & (col("Country name") == "Singapore"))

df_preg6.select("country name", "year", "Healthy life expectancy at birth").show()

+------------+----+--------------------------------+
|country name|year|Healthy life expectancy at birth|
+------------+----+--------------------------------+
|   Singapore|2019|                            77.1|
+------------+----+--------------------------------+

